In [4]:
import polars as pl
import datapane as dp
import altair as alt

In [5]:
reports = pl.scan_csv("../pipeline/data/processed/bfro_reports_geocoded.csv")

In [6]:
reports.head().collect()

observed,location_details,county,state,season,title,latitude,longitude,date,number,classification,hexid,temperature_high,temperature_mid,temperature_low,dew_point,humidity,cloud_cover,moon_phase,precip_intensity,precip_probability,precip_type,pressure,summary,conditions,uv_index,visibility,wind_bearing,wind_speed
str,str,str,str,str,str,f64,f64,str,i64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,str,str,f64,f64,f64,f64
"""hello. some ti…","""Off of rt 2 in…","""Ohio County""","""West Virginia""","""Winter""","""Report 10006: …",40.1333,-80.68192,"""12""",10006,"""Class A""","""8a2a86362d47ff…",39.3,34.7,32.1,29.5,81.5,97.5,0.02,0.031,100.0,"""[""rain""]""",1006.3,"""Cloudy skies t…","""Rain, Overcast…",null,9.1,231.4,18.3
"""I may have alr…","""The place we c…","""Grant County""","""Oregon""","""Summer""","""Report 10324: …",44.7751,-118.7717,"""4th""",10324,"""Class B""","""8a28aa64998fff…",89.7,68.6,46.5,45.4,52.7,25.0,0.0,0.472,100.0,"""[""rain""]""",1015.9,"""Partly cloudy …","""Rain, Partiall…",null,20.1,42.6,8.1
"""My husband and…","""There is a bac…","""Coffey County""","""Kansas""","""Fall""","""Report 10660: …",38.3908,-95.6491,"""9""",10660,"""Class A""","""8a26e528ac4fff…",49.5,46.5,39.6,44.2,91.7,83.6,0.85,0.283,100.0,"""[""rain""]""",1017.5,"""Partly cloudy …","""Rain, Partiall…",null,5.5,159.7,17.2
"""I live in hend…","""we are off of …","""Henderson Coun…","""Tennessee""","""Winter""","""Report 10740: …",35.80762,-88.30945,"""wed. the 9th""",10740,"""Class B""","""8a2648545ba7ff…",47.9,43.3,35.0,41.3,92.9,100.0,0.03,0.012,100.0,"""[""rain""]""",1018.1,"""Cloudy skies t…","""Rain, Overcast…",null,5.9,312.4,18.3
"""Around five ye…","""Longitude-87.9…","""Jasper County""","""Illinois""","""Spring""","""Report 10879: …",38.94564,-87.97622,"""?""",10879,"""Class B""","""8a264596a177ff…",80.7,70.3,60.9,62.1,77.4,16.3,0.09,null,0.0,null,1017.8,"""Clear conditio…","""Clear""",null,7.7,186.1,21.9


In [7]:
report_classifications = reports.groupby("classification").count().collect()
report_classifications

classification,count
str,u32
"""Class A""",2579
"""Class C""",33
"""Class B""",2668


In [8]:
from typing import Dict
classification_numbers: Dict[str, dp.BigNumber] = {}
for classification, count in report_classifications.iter_rows():
     classification_numbers[classification] = dp.BigNumber(
         heading=f"Number of {classification} Sightings",
         value=count
     )

In [20]:
report_classification_group = dp.Group(
    dp.Text("# Report Classifications"),
    dp.Group(
        classification_numbers["Class A"],
        classification_numbers["Class B"],
        classification_numbers["Class C"],
        dp.Text("""
        Class A reports involve clear sightings in circumstances where misinterpretation or misidentification of other animals can be ruled out with greater confidence.
        """),
        dp.Text("""
        Incidents where a possible sasquatch was observed at a great distance or in poor lighting conditions and incidents in any other circumstance that did not afford a clear view of the subject are considered Class B reports.
        """),
        dp.Text("""
        Most second-hand reports, and any third-hand reports, or stories with an untraceable sources, are considered Class C, because of the high potential for inaccuracy.
        """),
        columns=3
    ),
    columns=1
)

report_classification_group

In [101]:
from vega_datasets import data
states = alt.topo_feature(data.us_10m.url, feature="states")
background = alt.Chart(
    states
).mark_geoshape(
    fill="lightgray",
    stroke="white"
).project("albersUsa")

# lat_min / lat_max / lon_min / lon_max
# USA	United States	24.9493	49.5904	-125.0011	-66.9326

report_locations = reports.filter(
    pl.col("state").is_not_null() &
    pl.col("latitude").is_not_null() &
    pl.col("longitude").is_not_null() &
    (
        (
            # continental us
            (pl.col("latitude") > 24.943) &
            (pl.col("latitude") < 49.5904) &
            (pl.col("longitude") > -125.0011) &
            (pl.col("longitude") < -66.9326)
        ) |
        # or AK/HI
        (pl.col("state") == "Hawaii") |
        (pl.col("state") == "Alaska")
    )
).select(
    pl.col("latitude"),
    pl.col("longitude"),
    pl.col("classification")
).collect().to_pandas()

points = alt.Chart(report_locations).mark_circle(
    size=10,
).encode(
    longitude="longitude:Q", 
    latitude="latitude:Q",
    color=alt.Color("classification:N", title="Classification")
)

bfro_map = (background + points).properties(
    title="Bigfoot Sightings"
)

# bfro_map

In [11]:
reports_per_year = reports.filter(
    pl.col("date").str.contains("\d{4}-\d{2}-\d{2}")
).with_columns(
    sighting_year=pl.col("date").str.to_datetime("%Y-%m-%d").dt.year()
).groupby(
    "sighting_year",
    "classification"
).count().filter(
    pl.col("sighting_year") >= 2003
).collect()

reports_per_year_chart = alt.Chart(
    reports_per_year.to_pandas()
).mark_bar().encode(
    x=alt.X("sighting_year:O", title="Year"),
    y=alt.Y("count:Q", title="Number of Sightings"),
    color=alt.Color("classification:N", title="Classification"),
    tooltip=alt.Tooltip("count:Q")
).properties(
    title="Sightings by Year"
)

reports_per_year_chart

alt.Chart(...)

In [12]:
sightings_by_state = (
    reports
    .filter(pl.col("state").is_not_null())
    .groupby("state", "classification")
    .count()
    .sort(by="count", descending=True)
    .collect()
)


sightings_by_state_chart = (
    alt.Chart(sightings_by_state.to_pandas())
    .mark_bar()
    .encode(
        x=alt.X("state:N", title="State", sort="-y"),
        y=alt.Y("count:Q", title="Number of Sightings"),
        color=alt.Color("classification:N", title="Classification"),
        tooltip=alt.Tooltip("count:Q")
    )
    .properties(
        title="Sightings by State"
    )
)

sightings_by_state_chart


alt.Chart(...)

In [21]:
report_seasons = (
    reports
    .filter(pl.col("season").is_not_null())
    .groupby("season")
    .count()
    .collect()
)

report_seasons_dict: Dict[str, dp.BigNumber] = dict()

for season, count in report_seasons.iter_rows():
    report_seasons_dict[season] = dp.BigNumber(
        value=count,
        heading=f"Sightings in {season}"
    )

report_seasons_group = dp.Group(
    dp.Text("# Sightings by Season"),
    dp.Group(
        report_seasons_dict["Spring"],
        report_seasons_dict["Summer"],
        report_seasons_dict["Fall"],
        report_seasons_dict["Winter"],
        columns=4
    ),
    dp.Text(
        """
        Sightings are significantly higher in the summer and fall.
        This could be due to more daylight hours or friendlier temperatures in places 
        with higher sightings overall. Either way, it makes sense that there are more 
        reported sightings when the weather is nice, because people are more likely to 
        be outside.
        """
    ),
    columns=1
)

report_seasons_group

In [73]:
(
    reports
    .select(
        pl.col("temperature_mid"),
        pl.col("humidity"),
        pl.col("cloud_cover"),
        pl.col("precip_intensity"),
        pl.col("visibility"),
        pl.col("pressure")
    )
    .collect()
    .describe()
)

describe,temperature_mid,humidity,cloud_cover,precip_intensity,visibility,pressure
str,f64,f64,f64,f64,f64,f64
"""count""",5280.0,5280.0,5280.0,5280.0,5280.0,5280.0
"""null_count""",1270.0,1283.0,1315.0,1702.0,1338.0,1554.0
"""mean""",58.012219,68.593195,45.889206,0.094409,10.509614,1016.938701
"""std""",16.634994,15.686753,32.904563,0.307969,5.84781,6.146392
"""min""",-19.5,7.8,0.0,0.0,0.0,980.4
"""max""",96.1,100.0,100.0,7.468,74.6,1045.1
"""median""",59.85,70.9,43.6,0.0,9.7,1016.8
"""25%""",47.2,59.8,15.6,0.0,8.2,1013.2
"""75%""",70.9,79.8,75.0,0.035,9.9,1020.5


* temperature_mid ranges -19.5 to 96.1
* humidity ranges 7.8 to 100
* cloud cover ranges 0 to 100
* precip intensity ranges 0 to 7.468
* visibility ranges from 0 to 74.6
* pressure ranges from 980.4 to 1045.1

In [66]:
from typing import List, Optional
def add_bins(frame: pl.DataFrame, column: str, bins: List) -> pl.DataFrame:
    """ Adds the bin boundaries to the provided data frame, as
    a convenience for creating histograms.
    """
    cut_results = frame[column].cut(bins=bins, maintain_order=True)
    return frame.with_columns(
        cut_results["break_point"].alias(f"{column}_binned")
    )

def make_sighting_histogram_chart(
    frame: pl.LazyFrame,
    bins: List,
    column: str,
    title: Optional[str] = None
) -> alt.Chart:
    if title is None:
        title = column.title()
    
    frame_for_bins = (
        frame
        .filter(pl.col(column).is_not_null())
        .select(pl.col(column), pl.col("classification"))
        .collect()
    )
    
    frame_with_bins = add_bins(frame_for_bins, column, bins)
    
    frame_histogram = (
        frame_with_bins
        .groupby(f"{column}_binned", "classification")
        .count()
    )
    
    frame_histogram_chart = (
        alt.Chart(frame_histogram.to_pandas())
        .mark_bar()
        .encode(
            x=alt.X(f"{column}_binned:N", title=title),
            y=alt.Y("count:Q", title="Number of Sightings"),
            color=alt.Color("classification:N", title="Classification"),
            tooltip=alt.Tooltip("count:Q")
        )
        .properties(
            title=f"Sightings by {title}",
            width=alt.Step(50)
        )
    )
    
    return frame_histogram_chart

In [67]:
temperature_bins = list(range(-20, 101, 10))
temperature_histogram_chart = make_sighting_histogram_chart(
    reports,
    temperature_bins,
    "temperature_mid",
    title="Temperature"
)

temperature_histogram_chart

alt.Chart(...)

In [68]:
humidity_bins = list(range(0,101,10))
humidity_histogram_chart = make_sighting_histogram_chart(
    reports,
    humidity_bins,
    "humidity"
)

humidity_histogram_chart

alt.Chart(...)

In [69]:
cloud_cover_bins = list(range(0,101,10))
cloud_cover_histogram_chart = make_sighting_histogram_chart(
    reports,
    cloud_cover_bins,
    "cloud_cover",
    "Cloud Cover"
)

cloud_cover_histogram_chart

alt.Chart(...)

In [80]:
import numpy as np
precip_intensity_bins = np.arange(0.0, 8.25, 0.75).tolist()
precip_intensity_histogram_chart = make_sighting_histogram_chart(
    reports,
    precip_intensity_bins,
    "precip_intensity",
    title="Precipitation"
)
precip_intensity_histogram_chart

alt.Chart(...)

In [81]:
visibility_bins = np.arange(0.0, 82.5, 7.5)
visibility_histogram_chart = make_sighting_histogram_chart(
    reports,
    visibility_bins,
    "visibility",
    title="Visibility"
)
visibility_histogram_chart

alt.Chart(...)

In [85]:
pressure_bins = np.arange(980, 1060, 6.5).tolist()
pressure_histogram_chart = make_sighting_histogram_chart(
    reports,
    pressure_bins,
    "pressure"
)
pressure_histogram_chart

alt.Chart(...)

In [86]:
weather_group = dp.Group(
    temperature_histogram_chart,
    humidity_histogram_chart,
    cloud_cover_histogram_chart,
    precip_intensity_histogram_chart,
    visibility_histogram_chart,
    pressure_histogram_chart,
    columns=2
)

weather_group

In [102]:
report = dp.Blocks(
    bfro_map,
    report_classification_group,
    reports_per_year_chart,
    sightings_by_state_chart,
    report_seasons_group,
    weather_group
)

dp.save_report(report, "test.html")

App saved to ./test.html